#Nombre: Sebastián Urbina

# CC3001 Primavera 2020 Tarea 5
# Inserción en la raíz en un ABB
## Profesores
Sección 1 Benjamin Bustos • Sección 2 Jérémy Barbay • Sección 3 Patricio Poblete / Nelson Baloian

---

# CONTEXTO

El algoritmo usual de inserción en un ABB realiza una búsqueda infurctuosa para la nueva llave, y la coloca en el lugar de la hoja en donde termina la búsqueda. Se dice que es una *inserción en las hojas*.

Por contraposición, un algoritmo de *inserción en la raíz* deja la nueva llave como raíz del árbol resultante. En el apunte aparece un algoritmo de este tipo basado en *rotaciones*.

El objetivo de esta tarea es implementar inserción en la raíz de un ABB usando un algoritmo alternativo, basado en la operación ``cut``(corte).

Para comenzar, recordemos esta implementación del algoritmo de inserción usual (inserción en las hojas) adaptada del apunte:

In [61]:
class Nodoi:
    def __init__(self, izq, info, der):
        self.izq=izq
        self.info=info
        self.der=der
        if x<self.info:
            return Nodoi(self.izq.insert(x),self.info,self.der)
        else:
            return Nodoi(self.izq,self.info,self.der.insert(x))
        
    def __str__(self):
        return "("+self.izq.__str__()+str(self.info)+self.der.__str__()+")"

class Nodoe:
    def __init__(self):
        pass
    
    def insert(self,x):
        return Nodoi(Nodoe(),x,Nodoe())
    
    def __str__(self):
        return"☐"

class Arbol:
    def __init__(self,raiz=Nodoe()):
        self.raiz=raiz     
        
    def insert(self,x):
        self.raiz=self.raiz.insert(x)
    
    def __str__(self):        
        return self.raiz.__str__()

Probemos este algoritmo con una secuencia de inserciones:

In [43]:
lista=[4,9,6,2,7,1,8,3,5]
a=Arbol()
for x in lista:
    a.insert(x)
print(a)

(((☐1☐)2(☐3☐))4(((☐5☐)6(☐7(☐8☐)))9☐))


Para definir el algoritmo de inserción en la raíz, definimos primero la operación ``cut(x)``. La idea es la siguiente: dado un árbol $A$ y una llave $x$ que no está en el árbol, el resultado de ejecutar $\text{cut}(x)$ sobre el árbol $A$ es una tupla $(A_{<x},A_{>x})$, que contiene como primera componente el árbol que contiene solo las llaves que son menores que $x$, y similarmente la segunda componente es el árbol con las llaves mayores que $x$. Por ejemplo:

![T5-cut](https://github.com/ppoblete/CC3001-2020-2-Tareas/blob/master/T5-cut.png?raw=1)

La operación ``cut(x)`` se puede definir recursivamente:

Para el caso de un árbol no vacío con raíz $r$, con $x<r$:

![T5-cut2](https://github.com/ppoblete/CC3001-2020-2-Tareas/blob/master/T5-cut2.png?raw=1)

El caso $x>r$ es simétrico.

El caso de borde es que al hacer ``cut(x)``sobre un nodo ☐, el resultado es una tupla (☐,☐).

En base a esta operación ``cut(x)``, es fácil definir una operación ``root_insert(x)``. En efecto, para insertar una nueva llave $x$ en la raíz de un árbol $A$, se hace un corte con $x$ al árbol $A$, y luego se construye el árbol

![T5-cut3](https://github.com/ppoblete/CC3001-2020-2-Tareas/blob/master/T5-cut3.png?raw=1)

# INSTRUCCIONES

## Parte 1: ``cut(x)`` y ``root_insert(x)``

Modifique las clases provistas para agregar la operación ``cut(x)``y la operación ``root_insert(x)``. Escriba acá su código para las clases completas:

In [66]:
class Nodoi:
    def __init__(self, izq, info, der):
        self.izq=izq
        self.info=info
        self.der=der
    
    def cut(self, x):
        """
        Método que dado un valor x corta el ABB en dos subárboles y devuelve una tupla
         con dos árboles ABB de la forma(ABB<x,ABB>x)
        """
        if x < self.info:
            return self.izq.cut(x)[0], Nodoi(self.izq.cut(x)[1], self.info, self.der) #Si x<valor_padre entonces recorremos recursivamente el lado izquierdo del arbol, mientras el lado derecho se mantiene    
        return Nodoi(self.izq, self.info, self.der.cut(x)[0]), self.der.cut(x)[1] #Si x>valor_padre entonces recorremos recursivamente el lado derecho del arbol, mientras el lado izquierdo se mantiene

    def root_insert(self, x):
        return Nodoi(self.cut(x)[0], x, self.cut(x)[1]) #Cortamos el árbol y insertamos como padre al valor x
        
    def __str__(self):
        return "("+self.izq.__str__()+str(self.info)+self.der.__str__()+")"

class Nodoe:
    def __init__(self):
        pass
    
    def cut(self, x):
        return Nodoe(), Nodoe() #Caso base de cut(), cuando se llega a un nodo externo, se retorna una tupla de nodos externos

    def root_insert(self, x):
        return Nodoi(Nodoe(), x, Nodoe()) #Caso base de root_insert(), cuando llegamos a un nodo externo, retornamos un nodo interno con el valor insertado y dos hijos vacios

    def __str__(self):
        return"☐"

class Arbol:
    def __init__(self,raiz=Nodoe()):
        self.raiz=raiz     

    def cut(self, x):
        return self.raiz.cut(x)

    def root_insert(self, x):
        self.raiz = self.raiz.root_insert(x)
       
    def __str__(self):        
        return self.raiz.__str__()


## Parte 2: Evaluación experimental

In [64]:
#algunos ejemplos
a = Arbol(Nodoi(Nodoi(Nodoe(), 5, Nodoe()), 
                10,
                Nodoi(Nodoe(), 20, Nodoe())
                ))

b = Arbol(Nodoi(Nodoi(Nodoi(Nodoi(Nodoe(),1,Nodoe()) , 2, 
                            Nodoi(Nodoe(), 3, Nodoi(Nodoe(), 5, Nodoe()))) ,6,
                      Nodoi(Nodoe(), 7, Nodoi(Nodoe(),8,Nodoe()))),
                9,
                Nodoe()))
print(a.cut(6)[0], a.cut(6)[1])
print(b.cut(4)[0], b.cut(4)[1])

(☐5☐) (☐10(☐20☐))
((☐1☐)2(☐3☐)) (((☐5☐)6(☐7(☐8☐)))9☐)


A continuación, ejecute lo siguiente para probar su implementación:

In [65]:
lista=[4,9,6,2,7,1,8,3,5]
a=Arbol()
for x in reversed(lista):
    a.root_insert(x)
print(a)

(((☐1☐)2(☐3☐))4(((☐5☐)6(☐7(☐8☐)))9☐))


Si todo está bien, el resultado de insertar al lista reversa usando ``root_insert``debería ser igual al obtenido insertando la lista original usando ``insert``.